# TFRS - Ranking model

In [1]:
# !pip install tensorflow-recommenders google-cloud-aiplatform --user
! python3 -c "import google.cloud.aiplatform; print('aiplatform SDK version: {}'.format(google.cloud.aiplatform.__version__))"

aiplatform SDK version: 1.26.1


In [2]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [3]:
REGION           = 'us-central1' 
BUCKET_NAME      = 'matching-engine-content'    # location to store output
BUCKET_URI       = f'gs://{BUCKET_NAME}'

DATA_VERSION     = "v2-0-0" # version tag for dataflow pipeline

BUCKET_DATA_DIR  = 'spotify-data-regimes'
TRAIN_DIR_PREFIX = f'valid' 
VALID_DIR_PREFIX = f'valid'
CANDIDATE_PREFIX = f'candidates' 

In [4]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [5]:
import gc
gc.collect()

46

### imports 

In [6]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["CLOUD_ML_PROJECT_ID"] = PROJECT_ID

In [7]:
import json
import numpy as np
import pickle as pkl
import logging
import time
from pprint import pprint

# tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs

# configure GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Google Cloud SDKs
from google.cloud import aiplatform as vertex_ai
from google.cloud import storage

# GCS SDK client
storage_client = storage.Client(project=PROJECT_ID)

# ranking models
from src.ranking import tf_ranking_model as tfrm
from src.ranking import build_audio_ranker

# helpers / utils
# from util import feature_sets, train_utils, test_instances
from src.ranking import train_utils, feature_sets

ModuleNotFoundError: No module named 'ranking'

In [ ]:
# train_utils.full_parse
gpus

## Dataset

In [ ]:
!tree src/ranking

### create dataset objects

In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO

batch_size = 1024 #*16

#### train dataset

In [ ]:
train_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{TRAIN_DIR_PREFIX}/'):
    if '.tfrecords' in blob.name:
        train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

train_dataset = tf.data.Dataset.from_tensor_slices(train_files).prefetch(tf.data.AUTOTUNE)

# # ===================================================
# # baseline ranker feats
# # ===================================================
# train_dataset = train_dataset.interleave(
#     train_utils.full_parse,
#     cycle_length=tf.data.AUTOTUNE, 
#     num_parallel_calls=tf.data.AUTOTUNE,
#     deterministic=False,
# ).map(
#     feature_sets.parse_tfrecord, 
#     num_parallel_calls=tf.data.AUTOTUNE
# ).batch(
#     batch_size 
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(
#     options
# )

# ===================================================
# audio feats ranker
# ===================================================
train_dataset = train_dataset.interleave(
    train_utils.full_parse,
    num_parallel_calls=tf.data.AUTOTUNE,
    cycle_length=tf.data.AUTOTUNE, 
    deterministic=False,
).map(
    feature_sets.parse_audio_rank_tfrecord, 
    num_parallel_calls=tf.data.AUTOTUNE
).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
).with_options(
    options
)

# ===================================================
# cache dataset
# ===================================================
# train_dataset = train_dataset.cache()
# train_dataset

In [8]:
for x in train_dataset.batch(1).take(1):
    pprint(len(x))

NameError: name 'train_dataset' is not defined

#### validation set

In [13]:
valid_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{VALID_DIR_PREFIX}/'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(tf.data.AUTOTUNE)

# ===================================================
# baseline ranker feats
# ===================================================
# valid_dataset = valid_dataset.interleave(
#     train_utils.full_parse,
#     num_parallel_calls=tf.data.AUTOTUNE,
#     cycle_length=tf.data.AUTOTUNE, 
#     deterministic=False,
# ).map(
#     feature_sets.parse_tfrecord, 
#     num_parallel_calls=tf.data.AUTOTUNE
# ).batch(
#     batch_size
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(
#     options
# )

# ===================================================
# audio feats ranker
# ===================================================
valid_dataset = valid_dataset.interleave(
    train_utils.full_parse,
    num_parallel_calls=tf.data.AUTOTUNE,
    cycle_length=tf.data.AUTOTUNE, 
    deterministic=False,
).map(
    feature_sets.parse_audio_rank_tfrecord, 
    num_parallel_calls=tf.data.AUTOTUNE
).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
).with_options(
    options
)

# ===================================================
# cache dataset
# ===================================================
# valid_dataset = valid_dataset.cache() #1gb machine mem + 400 MB in candidate ds (src/two-tower.py)
# valid_dataset

In [14]:
for x in valid_dataset.batch(1).take(1):
    # pprint(x)
    pprint(len(x))

29


#### Create Candidates dataset

In [18]:
# candidate_files = []
# for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{CANDIDATE_PREFIX}/'):
#     if '.tfrecords' in blob.name:
#         candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

# candidate_dataset = tf.data.Dataset.from_tensor_slices(candidate_files)

# parsed_candidate_dataset = candidate_dataset.interleave(
#     train_utils.full_parse,
#     cycle_length=tf.data.AUTOTUNE, 
#     num_parallel_calls=tf.data.AUTOTUNE,
#     deterministic=False
# ).map(
#     feature_sets.parse_candidate_tfrecord_fn, 
#     num_parallel_calls=tf.data.AUTOTUNE
# ).with_options(
#     options
# )

# parsed_candidate_dataset = parsed_candidate_dataset.cache() #400 MB on machine mem
# # parsed_candidate_dataset

# # check dataset output
# for x in parsed_candidate_dataset.batch(1).take(1):
#     pprint(x)

### vocab dictionary

In [15]:
filehandler = open('vocab_dict.pkl', 'rb')
VOCAB_DICT = pkl.load(filehandler)
filehandler.close()

In [16]:
for keys in VOCAB_DICT:
    print(keys)

pl_name_src
track_name_pl
artist_name_pl
album_name_pl
artist_genres_pl
tracks_playlist_titles_pl
track_name_can
artist_name_can
album_name_can
artist_genres_can
track_pl_titles_can


## Build model

In [17]:
# from src.ranking import tf_ranking_model as tfrm
# from src.ranking import build_audio_ranker

In [18]:
# USE_CROSS_LAYER = True

SEED = 1234
MAX_PLAYLIST_LENGTH = 5
EMBEDDING_DIM = 128   
PROJECTION_DIM = 25  
USE_DROPOUT = True
DROPOUT_RATE = 0.33
MAX_TOKENS = 20000
LAYER_SIZES=[256,64]

LR = .1
opt = tf.keras.optimizers.Adagrad(LR)

In [19]:
# ===================================================
# baseline ranker feats
# ===================================================
# model = tfrm.TheRankingModel(
#     layer_sizes=LAYER_SIZES
#     , vocab_dict = VOCAB_DICT
#     , embedding_dim = EMBEDDING_DIM
#     , projection_dim = PROJECTION_DIM
#     , seed = SEED
#     , use_dropout = USE_DROPOUT
#     , dropout_rate = DROPOUT_RATE
#     , max_tokens = MAX_TOKENS
# )

# ===================================================
# audio feats ranker
# ===================================================
model = build_audio_ranker.TheRankingModel(
    layer_sizes=LAYER_SIZES
    , vocab_dict = VOCAB_DICT
    , embedding_dim = EMBEDDING_DIM
    , projection_dim = PROJECTION_DIM
    , seed = SEED
    , use_dropout = USE_DROPOUT
    , dropout_rate = DROPOUT_RATE
    , max_tokens = MAX_TOKENS
)

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [21]:
model.fit(valid_dataset, epochs=1) # valid_dataset | train_dataset

647/647 [==============================] - 320s 477ms/step - root_mean_squared_error: 0.4605 - loss: 0.2116 - regularization_loss: 0.0000e+00 - total_loss: 0.2116


In [22]:
eval_dict = model.evaluate(valid_dataset, return_dict=True)
eval_dict

647/647 [==============================] - 73s 110ms/step - root_mean_squared_error: 0.0022 - loss: 4.7898e-06 - regularization_loss: 0.0000e+00 - total_loss: 4.7898e-06


{'root_mean_squared_error': 0.002188949380069971,
 'loss': 4.020698270323919e-06,
 'regularization_loss': 0,
 'total_loss': 4.020698270323919e-06}

## save trained model

In [25]:
OUTPUT_BUCKET = 'gs://jt-tfrs-central-v4'
! gsutil mb -l $REGION $OUTPUT_BUCKET

Creating gs://jt-tfrs-central-v4/...


In [26]:
model_dir = f"{OUTPUT_BUCKET}/local-test-v2"

print(model_dir)

gs://jt-tfrs-central-v4/local-test-v1


In [27]:
tf.saved_model.save(model, export_dir=model_dir)

INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fa9fd0>, 139649424417872), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fa9fd0>, 139649424417872), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021731ea90>, 139649422462768), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021731ea90>, 139649422462768), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021733d1d0>, 139648235291152), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd6f10>, 139647318721280), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd7350>, 139647318721920), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd7350>, 139647318721920), {}).


INFO:tensorflow:Assets written to: gs://jt-tfrs-central-v4/local-test-v1/assets


INFO:tensorflow:Assets written to: gs://jt-tfrs-central-v4/local-test-v1/assets


### load saved model

In [28]:
loaded = tf.saved_model.load(model_dir)

In [29]:
loaded

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f01c5e98a90>

In [27]:
test_uri = 'gs://jt-tfrs-central-v4/test-rank-src-v5/run-20230629-083212/model-dir/ranking_model'
loaded_v2 = tf.saved_model.load(test_uri)

In [28]:
loaded_v2

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f74f5686ad0>

# Matching Engine - tests

In [52]:
from google.cloud import aiplatform as vertex_ai

# Vertex SDK 
vertex_ai.init(project=PROJECT_ID, location=REGION)

import time

In [53]:
INDEX_ENDPOINT_URI = 'projects/934903580331/locations/us-central1/indexEndpoints/4438886771006111744' # ann 50e

In [54]:
ME_index_endpoint = vertex_ai.MatchingEngineIndexEndpoint(INDEX_ENDPOINT_URI)
ME_index_endpoint

resource name: projects/934903580331/locations/us-central1/indexEndpoints/4438886771006111744

In [55]:
DEPLOYED_INDEX_ID = ME_index_endpoint.deployed_indexes[0].id
print(f"DEPLOYED_INDEX_ID: {DEPLOYED_INDEX_ID}")

ME_index_endpoint.deployed_indexes

DEPLOYED_INDEX_ID: deployedann_e2e_v1


[id: "deployedann_e2e_v1"
index: "projects/934903580331/locations/us-central1/indexes/68706282596466688"
create_time {
  seconds: 1681232232
  nanos: 385340000
}
private_endpoints {
  match_grpc_address: "10.41.2.5"
}
index_sync_time {
  seconds: 1687983053
  nanos: 643665000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

In [56]:
ENDPOINT_URI = 'projects/934903580331/locations/us-central1/endpoints/3302875755234459648'
model_endpoint = vertex_ai.Endpoint(ENDPOINT_URI)
model_endpoint

resource name: projects/934903580331/locations/us-central1/endpoints/3302875755234459648

In [57]:
print(model_endpoint.gca_resource.deployed_models[0])

id: "4612884486101663744"
model: "projects/934903580331/locations/us-central1/models/8584575572326219776"
display_name: "query-tower-sp-2tower-tfrs-e2e-v1-pipev4"
create_time {
  seconds: 1681222077
  nanos: 256958000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-16"
  }
  min_replica_count: 1
  max_replica_count: 1
}
service_account: "notebooksa@hybrid-vertex.iam.gserviceaccount.com"
model_version_id: "1"



In [58]:
# # MAX_PLAYLIST_LENGTH --> 5
TEST_INSTANCE_5 = test_instances.TEST_INSTANCE_5
TEST_INSTANCE_5

{'album_name_can': 'Capoeira Electronica',
 'album_name_pl': ['Odilara',
  'Capoeira Electronica',
  'Capoeira Ultimate',
  'Festa Popular',
  'Capoeira Electronica'],
 'album_uri_can': 'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR',
 'album_uri_pl': ['spotify:album:4Y8RfvZzCiApBCIZswj9Ry',
  'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR',
  'spotify:album:55HHBqZ2SefPeaENOgWxYK',
  'spotify:album:150L1V6UUT7fGUI3PbxpkE',
  'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR'],
 'artist_followers_can': 5170.0,
 'artist_genres_can': 'capoeira',
 'artist_genres_pl': ['samba moderno',
  'capoeira',
  'capoeira',
  'NONE',
  'capoeira'],
 'artist_name_can': 'Capoeira Experience',
 'artist_name_pl': ['Odilara',
  'Capoeira Experience',
  'Denis Porto',
  'Zambe',
  'Capoeira Experience'],
 'artist_pop_can': 24.0,
 'artist_pop_pl': [4.0, 24.0, 2.0, 0.0, 24.0],
 'artist_uri_can': 'spotify:artist:5SKEXbgzIdRl3gQJ23CnUP',
 'artist_uri_pl': ['spotify:artist:72oameojLOPWYB7nB8rl6c',
  'spotify:artist:5SKEXbgzIdRl3gQJ23CnUP',


In [60]:
playlist_emb = model_endpoint.predict([TEST_INSTANCE_5])
playlist_emb

Prediction(predictions=[[2.13599348, 0.665572762, 0.306572795, -0.642993093, 0.905611515, -0.451913774, -1.04571533, -1.47509861, 0.667647898, 1.392694, -0.580560744, 2.1635685, 1.15477574, 0.890988827, 0.987143219, -0.344569176, -0.727035463, -0.337408423, -1.34015453, -1.03236663, -0.695001364, -0.106723189, 0.0395636633, 1.66761637, 0.981151044, 0.879824638, 0.642675459, 2.18829846, -1.37034965, 0.116360746, -0.356807619, -0.00551904133, -1.61825109, 1.00126624, -0.543757379, -0.640756607, 0.584731758, -1.26177561, 1.33843577, 0.260981917, 0.691716552, 0.555235922, 0.835755944, 0.242919102, 0.508965909, -0.551774085, -0.752674818, 1.07970881, -0.278400809, 1.20874846, 0.634759903, 0.753106713, -0.331600159, -0.451509297, -0.44077149, -0.706078768, 0.518127501, 0.231186, 1.1358285, -0.811849892, -0.790187061, 0.0396273322, 0.101898566, -1.96892595, 0.238233238, -0.289123625, 0.353820384, 0.371209621, 1.90042961, -0.360776, 0.874980628, 0.49637109, -0.448792934, -0.249126494, -1.58931

In [68]:
len(playlist_emb.predictions[0])

128

In [72]:
import numpy as np 

original = playlist_emb.predictions[0]
opposite = (original * np.ones(128) * -1)
opposite

array([-2.13599348, -0.66557276, -0.3065728 ,  0.64299309, -0.90561152,
        0.45191377,  1.04571533,  1.47509861, -0.6676479 , -1.392694  ,
        0.58056074, -2.1635685 , -1.15477574, -0.89098883, -0.98714322,
        0.34456918,  0.72703546,  0.33740842,  1.34015453,  1.03236663,
        0.69500136,  0.10672319, -0.03956366, -1.66761637, -0.98115104,
       -0.87982464, -0.64267546, -2.18829846,  1.37034965, -0.11636075,
        0.35680762,  0.00551904,  1.61825109, -1.00126624,  0.54375738,
        0.64075661, -0.58473176,  1.26177561, -1.33843577, -0.26098192,
       -0.69171655, -0.55523592, -0.83575594, -0.2429191 , -0.50896591,
        0.55177409,  0.75267482, -1.07970881,  0.27840081, -1.20874846,
       -0.6347599 , -0.75310671,  0.33160016,  0.4515093 ,  0.44077149,
        0.70607877, -0.5181275 , -0.231186  , -1.1358285 ,  0.81184989,
        0.79018706, -0.03962733, -0.10189857,  1.96892595, -0.23823324,
        0.28912362, -0.35382038, -0.37120962, -1.90042961,  0.36

In [73]:
original

[2.13599348,
 0.665572762,
 0.306572795,
 -0.642993093,
 0.905611515,
 -0.451913774,
 -1.04571533,
 -1.47509861,
 0.667647898,
 1.392694,
 -0.580560744,
 2.1635685,
 1.15477574,
 0.890988827,
 0.987143219,
 -0.344569176,
 -0.727035463,
 -0.337408423,
 -1.34015453,
 -1.03236663,
 -0.695001364,
 -0.106723189,
 0.0395636633,
 1.66761637,
 0.981151044,
 0.879824638,
 0.642675459,
 2.18829846,
 -1.37034965,
 0.116360746,
 -0.356807619,
 -0.00551904133,
 -1.61825109,
 1.00126624,
 -0.543757379,
 -0.640756607,
 0.584731758,
 -1.26177561,
 1.33843577,
 0.260981917,
 0.691716552,
 0.555235922,
 0.835755944,
 0.242919102,
 0.508965909,
 -0.551774085,
 -0.752674818,
 1.07970881,
 -0.278400809,
 1.20874846,
 0.634759903,
 0.753106713,
 -0.331600159,
 -0.451509297,
 -0.44077149,
 -0.706078768,
 0.518127501,
 0.231186,
 1.1358285,
 -0.811849892,
 -0.790187061,
 0.0396273322,
 0.101898566,
 -1.96892595,
 0.238233238,
 -0.289123625,
 0.353820384,
 0.371209621,
 1.90042961,
 -0.360776,
 0.874980628,
 0

In [74]:
DEPLOYED_ANN_INDEX_ID = 'deployedann_e2e_v1'

In [78]:
ANN_response = ME_index_endpoint.match(
    deployed_index_id=DEPLOYED_ANN_INDEX_ID,
    queries=[original],
    num_neighbors=10
)
# ANN_response

In [79]:
ANN_response

[[MatchNeighbor(id='spotify:track:5ujxgPve1zpywKhdhprjd7', distance=-68.44378662109375),
  MatchNeighbor(id='spotify:track:4IkKAbJxc6XMt6dP0Wxn9q', distance=-69.26606750488281),
  MatchNeighbor(id='spotify:track:77sA6Lhq1aHyeiDn0Ezynk', distance=-69.32096862792969),
  MatchNeighbor(id='spotify:track:3oTZEx1KVuCaGnDwOaluRv', distance=-69.49063873291016),
  MatchNeighbor(id='spotify:track:2GclgkNkaCOgauDyGSgn5O', distance=-69.74806213378906),
  MatchNeighbor(id='spotify:track:4djyU5pTSNYq2ZuBFRia7I', distance=-69.94599914550781),
  MatchNeighbor(id='spotify:track:0nhk50UU12qwXp3ap9KZH5', distance=-70.12556457519531),
  MatchNeighbor(id='spotify:track:6wL1b6VtUdmddjXzpURDBr', distance=-70.27442932128906),
  MatchNeighbor(id='spotify:track:7oKRbhVq8AqvnfIrSmh6bN', distance=-70.29933166503906),
  MatchNeighbor(id='spotify:track:2wb8SJRrCx4k7bgTk0HPXU', distance=-70.32716369628906)]]

In [80]:
ANN_response_v2 = ME_index_endpoint.match(
    deployed_index_id=DEPLOYED_ANN_INDEX_ID,
    queries=[opposite],
    num_neighbors=10
)
# ANN_response

In [81]:
ANN_response_v2

[[MatchNeighbor(id='spotify:track:2vsNqssAT6DwTr8XMtkvuI', distance=120.07003784179688),
  MatchNeighbor(id='spotify:track:6FrqntvCqYk6pTloZ5NGcS', distance=119.92662811279297),
  MatchNeighbor(id='spotify:track:1GzygBcsXQW5HVEvIky76a', distance=118.85808563232422),
  MatchNeighbor(id='spotify:track:0rcB5TG1EeMRkZFJDkDtGF', distance=118.80642700195312),
  MatchNeighbor(id='spotify:track:4nr2whZo0rB6blw1BaacZz', distance=117.85485076904297),
  MatchNeighbor(id='spotify:track:2XIuiCqPKof2hJfNzLJ9uW', distance=117.66204833984375),
  MatchNeighbor(id='spotify:track:20EFdLOP1Rm37ha8BLbOYe', distance=117.60924530029297),
  MatchNeighbor(id='spotify:track:0atv7V8rRNHYY9hroCQddh', distance=117.54971313476562),
  MatchNeighbor(id='spotify:track:5JELaD59SYjlNvl47UggH3', distance=117.53295135498047),
  MatchNeighbor(id='spotify:track:5WAyXO575REnpzp56Lm3m9', distance=117.46258544921875)]]